In [1]:
from reid_functions import *
import shutil
import time
import json

cfg = Config()
TVT, TMO = set_devices((0,))
model = Model(last_conv_stride=cfg.last_conv_stride)
model_w = DataParallel(model)
tri_loss = TripletLoss(margin=cfg.margin)

optimizer = optim.Adam(model.parameters(),
                     lr=cfg.base_lr,
                     weight_decay=cfg.weight_decay)

modules_optims = [model, optimizer]

map_location = (lambda storage, loc: storage)
sd = torch.load(cfg.model_weight_file, map_location=map_location)
load_state_dict(model, sd)

ext = ExtractFeature(model=model_w,TVT=TVT)
root_directory = '/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/Dynamic_Database/'
db_name_list, db_feature_list = extract_DB_features(ext,cfg,root_directory)

querrypath = "/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/query5/"
temppath = "/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/temp/"


/home/bmsknight/anaconda3/envs/pytorch/lib/python2.7/site-packages/scikit_learn-0.20.0-py2.7-linux-x86_64.egg/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Keys not found in source state_dict: 
	 base.layer2.0.bn1.num_batches_tracked
	 base.layer4.1.bn3.num_batches_tracked
	 base.layer4.0.bn1.num_batches_tracked
	 base.layer3.5.bn1.num_batches_tracked
	 base.layer3.2.bn2.num_batches_tracked
	 base.layer1.0.bn1.num_batches_tracked
	 base.layer1.1.bn2.num_batches_tracked
	 base.bn1.num_batches_tracked
	 base.layer3.1.bn3.num_batches_tracked
	 base.layer2.0.bn3.num_batches_tracked
	 base.layer2.0.downsample.1.num_batches_tracked
	 base.layer4.0.bn2.num_batches_tracked
	 base.layer2.2.bn3.num_batches_tracked
	 base.layer3.4.bn2.num_batches_tracked
	 base.layer3.4.bn3.num_batches_tracked
	 base.layer1.0.bn3.num_batches_tracked
	 base.layer1.1.bn1.num_batches_tracked
	 base.layer4.0.bn3.num_batches_tracked
	 base.layer1.1.bn3.num_batches_tracked
	 base.layer3.0.bn3.num_batches_tracked
	 base.layer2.3.bn1.num_batches_tracked
	 base.layer3.2.bn3.num_batches_tracked
	 base.layer3.0.bn2.num_batches_tracked
	 base.layer4.2.bn3.num_batches_tracked
	 

In [2]:
start = time.time()
n = 450
previous_frame = 0
attendance = {}
for k in range(20):
    copy_n_frames(n,previous_frame,querrypath,temppath)
    querry_name_list, querry_feature_list = extract_querry_features(ext, cfg, querrypath =temppath)
    q_q_dist = compute_dist(querry_feature_list,querry_feature_list, type='cosine')
    g_g_dist = compute_dist(db_feature_list,db_feature_list, type='cosine') 
    q_g_dist_cos = compute_dist(querry_feature_list,db_feature_list, type='cosine')
    q_g_dist_cos = np.ones_like(q_g_dist_cos) - q_g_dist_cos
    q_q_dist = np.ones_like(q_q_dist) - q_q_dist
    g_g_dist = np.ones_like(g_g_dist) - g_g_dist
    reranked_list = re_ranking(q_g_dist_cos, q_q_dist, g_g_dist)
    querry_name_list = querry_name_list.astype(int)
    voted_dictionary, voted_name_list = vote_for_person(reranked_list,querry_name_list,db_name_list)
    print k
    for item in voted_dictionary:
        print( item, voted_dictionary[item])
    display_dictionary, coordinate_dictionary, centroid_dictionary = find_valid_persons(voted_dictionary,voted_name_list,querry_name_list,n/10)
    
    for item in display_dictionary:
        print( item, display_dictionary[item])
    for item in coordinate_dictionary:
        if item in attendance.keys():
            attendance[item] = attendance[item]+1
        else:
            attendance[item] = 1
        print( item, coordinate_dictionary[item])
        
    with open('centroids'+str(k)+'.txt', 'w') as file:
        file.write(json.dumps(centroid_dictionary))
    
    image = cv2.imread('./frame/frame'+str(n+previous_frame)+'.jpg')
    for key in coordinate_dictionary:
        x1 = int(coordinate_dictionary[key][3])
        x2 = int(coordinate_dictionary[key][2])
        y1 = int(coordinate_dictionary[key][1])
        y2 = int(coordinate_dictionary[key][0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 165, 255), 2)
        cv2.putText(image, key, (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.imwrite('out'+str(k)+'.jpg',image)

    shutil.rmtree(temppath[:-1])
    os.mkdir(temppath[:-1])
    previous_frame = previous_frame + n
total_time = time.time()-start

for item in attendance:
    attendance[item] = attendance[item]*100/(k+1)
    print( item, attendance[item])
        
with open('attendance.txt', 'w') as file:
        file.write(json.dumps(attendance))
print (total_time)        

0
(0, array([ 44.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]))
(1, array([  0.,   0.,   0.,  44.,   0.,   0.,   0.,   0.,   0.,   0.]))
(2, array([  0.,   0.,   2.,   0.,   0.,   0.,   0.,  42.,   0.,   0.]))
(3, array([  0.,   0.,   0.,   0.,  44.,   0.,   0.,   0.,   0.,   0.]))
(4, array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  44.]))
(5, array([  0.,   0.,   0.,   0.,   0.,  44.,   0.,   0.,   0.,   0.]))
(6, array([  0.,   0.,   0.,   0.,   0.,   0.,  44.,   0.,   0.,   0.]))
(7, array([  0.,  44.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]))
(8, array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  44.,   0.]))
(9, array([  0.,   0.,  42.,   0.,   0.,   0.,   0.,   2.,   0.,   0.]))
(0, 'thuvaragan')
(1, 'athavan')
(2, 'madhushan')
(3, 'kabitha')
(4, 'thivakaran')
(5, 'harishanth')
(6, 'jathushan')
(7, 'piriyanthan')
(8, 'nilakshan')
(9, 'neerajan')
('thuvaragan', array([  984.5,   227. ,  1794.5,  1433. ]))
('nilakshan', array([ 451. ,   62. ,  